In [14]:
import geopandas as gpd
import os
import utils.analysis as an

# Zonas Medellin

In [15]:
zo = ['moravia_LH','floresta_HH','aguacatala_HL']
# z = zo[0]
for z in zo:
    print(f"Procesando zona: {z}")
    nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
    blocks = gpd.read_file(f'../data/output/shape/blocks/medellin_blocks.shp')

    blocks = blocks[['MANZ_CCNCT','pop','geometry']]
    nodes = nodes.to_crs('epsg:32618')
    blocks = blocks.to_crs('epsg:32618')

    # Create the voronoi polygons
    nodes = nodes.set_index('osmid')
    voronoi = an.voronoi_polygons(nodes)
    voronoi.to_crs('epsg:4326').to_file(f'../data/output/shape/physical_variables/population/voronoi/{z}/{z}_voronoi.shp')

    # Assing the population to each voronoi polygon
    blocks = blocks.set_index('MANZ_CCNCT')
    voronoi = an.assing_blocks_attribute_to_voronoi(blocks, voronoi, 'pop')


    # Calculate the population density
    voronoi['den_pop'] = voronoi['pop']/(voronoi.area/10000)


    # Merge the population with the nodes
    nodes = nodes.merge(voronoi[['osmid','den_pop']].set_index('osmid'), left_index=True, right_index=True, how='left')

    # Assing the population to the edges with an average
    for i in edges.index:
        u = edges.loc[i,'u']
        v = edges.loc[i,'v']
        edges.loc[i,'den_pop'] = (nodes.loc[u,'den_pop']+ nodes.loc[v,'den_pop'])/2
        
    # Export results
    nodes = nodes.reset_index()
    nodes = nodes.to_crs('epsg:4326')
    nodes.to_file(f'../data/output/shape/physical_variables/population/{z}/{z}_population_nodes.shp')
    edges.to_file(f'../data/output/shape/physical_variables/population/{z}/{z}_population_edges.shp')

Procesando zona: moravia_LH
Procesando zona: floresta_HH
Procesando zona: aguacatala_HL
